# Spotify Dataset 1922-2021
https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=tracks.csv


In [ ]:
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import psycopg2 as pg
from psycopg2 import Error
import sqlalchemy as sqla
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
# variáveis de conexão com o banco de dados
db = 'utfpr'
dbschema = 'pi3'  
dbuser = 'utfpr'
dbpass = 'postutfpr@8020'
dbhost = 'localhost'
dbport = '5432'

conn = sqla.create_engine(
      f'postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}:{dbport}/{db}'
    , connect_args={'options': '-csearch_path={}'.format(dbschema)}
)

In [3]:
# função genérica para realizar o put dos datasets direto no banco postgresql
# exemplo: put_sql_database(dataframe_criado, tabela_a_ser_criada_no_banco, variavel_da_conexao)
def put_sql_database(df_put, tabela, conexao):
    df_put.to_sql(
        name=tabela,
        con=conexao,
        index=False,
        if_exists='replace'
    )

In [4]:
# variável com o caminho para os arquivos CSV´s
pasta = 'archive'

# listando os arquivos do diretório archive
caminhos = [os.path.join(pasta, nome) for nome in os.listdir(pasta)]
arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
csvs = [arq for arq in arquivos if arq.lower().endswith(".csv")]

In [5]:
import os
x = [i[2] for i in os.walk('archive')]
csvs=[]
for t in x:
    for f in t:
        csvs.append(f)

In [ ]:
# gravando no banco de dados cada arquivo csv será uma tabela no banco
for csv in csvs: 
    tabela = os.path.splitext(csv)[0]  # nome do arquivo sem a extensão
    
    df = pd.read_csv(pasta +'/'+ csv)  # cria um dataframe com o nome do arquivo
    
    features = df.select_dtypes(include='number').columns  # seleciona somente as features numéricas do DataFrame, já explorado anteriormente  o pandas reconhece muito bem a tipagem destes arquivos
    
    list_caracteristicas = []  # lista vazia que irá receber as características

    for i in df.index:  # percorre as features e popula a lista vazia
        list_caracteristicas.append(list(df[features].loc[i]))
    
    df['vetor_caracteristicas'] = list_caracteristicas 
    df['vetor_caracteristicas'].replace(['[', ']'], ['{','}'])
    
    put_sql_database(df, tabela, conn)
    psql.execute(f'ALTER TABLE {dbschema}.{tabela} ALTER COLUMN vetor_caracteristicas TYPE _float8 USING vetor_caracteristicas::_float8;', conn);